In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


In [2]:
#Load the csv exported in Part I to a DataFrame
path = "output_data/clean_city_data.csv"

# Reading csv file and creating new dataframe:
city_df = pd.read_csv(path)

# Cleaning dataframe:
cleaned_df = city_df.dropna()
cleaned_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Joshīmath,30.5667,79.5667,71.92,94,94,2.01,IN,1660844964
1,Vao,-22.6667,167.4833,68.31,56,20,10.51,NC,1660844967
2,Ushuaia,-54.8000,-68.3000,42.46,65,20,19.57,AR,1660844811
3,Hobyo,5.3505,48.5268,76.84,83,85,25.97,SO,1660844971
4,Bredasdorp,-34.5322,20.0403,54.77,91,81,13.53,ZA,1660844974
...,...,...,...,...,...,...,...,...,...
542,Penhold,52.1334,-113.8687,74.84,54,0,10.92,CA,1660846343
543,Chittarkonda,18.1167,82.0833,74.95,92,100,1.21,IN,1660846346
544,Flinders,-34.5833,150.8552,58.05,85,99,1.99,AU,1660846348
545,Patacamaya,-17.2333,-67.9167,53.13,5,0,1.79,BO,1660846351


In [3]:
#configure g maps 

gmaps.configure(api_key=g_key)

In [4]:
#Use the Lat and Lng as locations and Humidity as the weight.

locations = cleaned_df[["Lat", "Lng"]]
humidity = cleaned_df["Humidity"]

In [5]:
# Add Heatmap layer to map:
heat = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity)

heat.add_layer(heatmap_layer)
heat

Figure(layout=FigureLayout(height='420px'))

In [6]:
#Narrow down the cities to fit my ideal weather conditions.

#no cloudiness:
nocloud_df = cleaned_df[cleaned_df["Cloudiness"] == 0]

# A max temperature lower than 80 degrees but higher than 65:
temp_df = nocloud_df[(nocloud_df["Max Temp"] > 65) & (nocloud_df["Max Temp"] < 80)]

# Wind speed less than 10 mph:
lowwind_df = temp_df[temp_df["Wind Speed"] < 10]

#humidity lower than 70 
humidity_df = lowwind_df[lowwind_df["Humidity"] < 70]

humidity_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
33,Yangi Marg`ilon,40.4272,71.7189,77.72,31,0,6.91,UZ,1660845048
201,Dustlik,40.5247,68.0358,71.60,20,0,9.19,UZ,1660845480
211,Vallenar,-28.5708,-70.7581,76.87,15,0,3.94,CL,1660845504
232,Korla,41.7597,86.1469,76.60,37,0,4.47,CN,1660845555
321,Yumen,40.2833,97.2000,69.60,26,0,7.05,CN,1660845779
507,Itum-Kale,42.7356,45.5758,70.70,33,0,7.92,RU,1660846256


In [7]:
#Store into variable named hotel_df.
#add hotel name column 

hotel_df = humidity_df.reset_index(drop=True)
hotel_df["Hotel Name"] = ""


In [8]:
#set parameters 

params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key }

In [12]:
# Use the latitude and longitude we recovered to identify hotels
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude
    lat = row["Lat"]
    lng = row["Lng"]

In [25]:
# Loop through to identify hotels:
for index, row in hotel_df.iterrows():
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    params['location'] = f'{lat},{lng}'
    
    hotel_data = requests.get(base_url, params=params).json()
    hotel_df.loc[index, "Hotel Name"] = hotel_data['results'][0]['name']
        
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Yangi Marg`ilon,40.4272,71.7189,77.72,31,0,6.91,UZ,1660845048,Hotel Front Castle
1,Dustlik,40.5247,68.0358,71.60,20,0,9.19,UZ,1660845480,Hotel Front Castle
2,Vallenar,-28.5708,-70.7581,76.87,15,0,3.94,CL,1660845504,Hotel Front Castle
3,Korla,41.7597,86.1469,76.60,37,0,4.47,CN,1660845555,Hotel Front Castle
4,Yumen,40.2833,97.2000,69.60,26,0,7.05,CN,1660845779,Hotel Front Castle
5,Itum-Kale,42.7356,45.5758,70.70,33,0,7.92,RU,1660846256,Hotel Front Castle


In [26]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [27]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

heat.add_layer(marker_layer)
# Display figure
heat

Figure(layout=FigureLayout(height='420px'))